In [1]:
#
import os
import copy
import sys
import warnings
import matplotlib
from pathlib import Path
from typing import Dict

#
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

# FinRL
from finrl.config_tickers import DOW_30_TICKER

from rl.experiments.common.utils import ignore_warnings

In [2]:
ignore_warnings()

In [3]:
#
import numpy as np
import pandas as pd
import tqdm

#
from common.utils import now_time
from configuration.settings import ProjectDir, ExperimentDir
from rl.data.CompanyInfo import CompanyInfo
from rl.experiments._3_key_metrics_dow30.train import base_cols, data_cols, ratios_cols, dataset_name

In [4]:
#
prj_dir = ProjectDir(__file__)
exp_dir = ExperimentDir(root=Path(os.getcwd()))
exp_dir.check_and_create_dirs()

#
TICKERS = copy.deepcopy(DOW_30_TICKER)
TICKERS.remove("DOW")  # TODO: I don't have all necessary data

In [5]:
TICKERS

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS']

In [6]:
print(prj_dir.root)
print(exp_dir.root)

/Users/zlapik/my-drive-zlapik/0-todo/ai-investing
/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/src/rl/experiments/_3_key_metrics_dow30


In [7]:
print(base_cols)
print(data_cols)
print(ratios_cols)

['date', 'tic']
['open', 'high', 'low', 'close', 'volume']
['operatingProfitMargin', 'netProfitMargin', 'returnOnAssets', 'returnOnEquity', 'currentRatio', 'quickRatio', 'cashRatio', 'inventoryTurnover', 'receivablesTurnover', 'payablesTurnover', 'debtRatio', 'debtEquityRatio', 'priceEarningsRatio', 'priceBookValueRatio', 'dividendYield']


In [8]:
# Load All Initial Tickers Data
tickers_data: Dict[str, CompanyInfo] = dict()
for tic in TICKERS:
    data = dict(symbol=tic)
    files = copy.deepcopy(CompanyInfo.Names.list())
    files.remove("symbol")
    for f in files:
        tic_file = prj_dir.data.tickers.joinpath(tic).joinpath(f + ".csv")
        if tic_file.exists():
            data[f] = pd.read_csv(tic_file, index_col=0)
        else:
            raise FileExistsError(f"File not exists: {tic_file}")
    tickers_data[tic] = CompanyInfo(**data)

In [9]:
print(tickers_data.keys())
# print(tickers_data.values())

dict_keys(['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS'])


In [10]:
dataset = pd.DataFrame()

# Merge tickers information into one pd.Dataframe
# for k, v in [("DIS", tickers_data["DIS"])]:
for k, v in tickers_data.items():
    # Prices
    data = v.data_detailed[data_cols]
    data.insert(0, "tic", k)

    # Fill before or forward
    data = data.fillna(method="bfill")
    data = data.fillna(method="ffill")

    # Ratios
    ratios = v.key_metrics.transpose()

    # Fill 0, where Nan/np.inf
    ratios = ratios.fillna(0)
    ratios = ratios.replace(np.inf, 0)

    #
    merge = pd.merge(data, ratios, how="outer", left_index=True, right_index=True)
    filled = merge.fillna(method="bfill")
    filled = filled.fillna(method="ffill")
    clean = filled.drop(filled[~filled.index.str.contains("\d{4}-\d{2}-\d{2}")].index)
    dataset = pd.concat([clean, dataset])

dataset.insert(0, "date", dataset.index)

In [11]:
assert dataset.isna().any().any() == False  # Can't be any Nan/np.inf values

In [12]:
dataset

,date,tic,open,high,low,close,volume,period,revenuePerShare,netIncomePerShare,...,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare
1962-01-02,1962-01-02,DIS,0.092908,0.096026,0.092908,0.092908,817400.0,Q1,0.7932324623088348,0.10738995608341934,...,0.0,909050000,65.966464834653,133.67062107466853,23.876482903000696,1.3643295911883075,0.6732967893500391,3.7693993862341078,0.05439630703970556,-0.07130594561043554
1962-01-03,1962-01-03,DIS,0.092908,0.094467,0.092908,0.094155,778500.0,Q1,0.7932324623088348,0.10738995608341934,...,0.0,909050000,65.966464834653,133.67062107466853,23.876482903000696,1.3643295911883075,0.6732967893500391,3.7693993862341078,0.05439630703970556,-0.07130594561043554
1962-01-04,1962-01-04,DIS,0.094155,0.094467,0.093532,0.094155,934200.0,Q1,0.7932324623088348,0.10738995608341934,...,0.0,909050000,65.966464834653,133.67062107466853,23.876482903000696,1.3643295911883075,0.6732967893500391,3.7693993862341078,0.05439630703970556,-0.07130594561043554
1962-01-05,1962-01-05,DIS,0.094155,0.094779,0.093844,0.094467,934200.0,Q1,0.7932324623088348,0.10738995608341934,...,0.0,909050000,65.966464834653,133.67062107466853,23.876482903000696,1.3643295911883075,0.6732967893500391,3.7693993862341078,0.05439630703970556,-0.07130594561043554
1962-01-08,1962-01-08,DIS,0.094467,0.095714,0.092285,0.094155,1245600.0,Q1,0.7932324623088348,0.10738995608341934,...,0.0,909050000,65.966464834653,133.67062107466853,23.876482903000696,1.3643295911883075,0.6732967893500391,3.7693993862341078,0.05439630703970556,-0.07130594561043554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-12,2022-12-12,AXP,154.260000,157.760000,154.000000,157.310000,2773675.0,Q3,18.060160427807485,2.5120320855614975,...,11196000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.07848788638262323,-0.5922459893048129
2022-12-13,2022-12-13,AXP,160.620000,161.550000,156.160000,157.520000,3110000.0,Q3,18.060160427807485,2.5120320855614975,...,11196000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.07848788638262323,-0.5922459893048129
2022-12-14,2022-12-14,AXP,157.570000,158.620000,153.840000,154.110000,3554804.0,Q3,18.060160427807485,2.5120320855614975,...,11196000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.07848788638262323,-0.5922459893048129
2022-12-15,2022-12-15,AXP,151.650000,152.280000,146.470000,150.220000,4282628.0,Q3,18.060160427807485,2.5120320855614975,...,11196000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.07848788638262323,-0.5922459893048129


In [13]:
# Take only those dates where we have data for all stock in each day
max_size = dataset.groupby("date").size().unique().max()
_d = dataset.groupby("date").size()
binary = _d.values == 29

print(_d.size)
print(binary.size)

latest_date = _d[binary].index[0]
dataset = dataset[dataset["date"] > latest_date]

15347
15347


In [14]:
dataset

,date,tic,open,high,low,close,volume,period,revenuePerShare,netIncomePerShare,...,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare
2008-03-20,2008-03-20,DIS,31.280001,31.980000,31.250000,31.900000,13802300.0,Q3,4.861052631578947,0.6757894736842105,...,0.0,1291000000,55.241446513642266,65.43866943866944,12.374220374220373,1.6292115011465866,1.3753335874952344,7.273185483870968,0.039171420726684766,-0.18578947368421053
2008-03-24,2008-03-24,DIS,32.000000,32.160000,31.799999,32.040001,10536000.0,Q3,4.861052631578947,0.6757894736842105,...,0.0,1291000000,55.241446513642266,65.43866943866944,12.374220374220373,1.6292115011465866,1.3753335874952344,7.273185483870968,0.039171420726684766,-0.18578947368421053
2008-03-25,2008-03-25,DIS,31.990000,32.240002,31.840000,32.080002,8547300.0,Q3,4.861052631578947,0.6757894736842105,...,0.0,1291000000,55.241446513642266,65.43866943866944,12.374220374220373,1.6292115011465866,1.3753335874952344,7.273185483870968,0.039171420726684766,-0.18578947368421053
2008-03-26,2008-03-26,DIS,31.910000,32.040001,31.530001,31.760000,9219700.0,Q3,4.861052631578947,0.6757894736842105,...,0.0,1291000000,55.241446513642266,65.43866943866944,12.374220374220373,1.6292115011465866,1.3753335874952344,7.273185483870968,0.039171420726684766,-0.18578947368421053
2008-03-27,2008-03-27,DIS,31.879999,31.959999,31.309999,31.379999,9569900.0,Q3,4.861052631578947,0.6757894736842105,...,0.0,1291000000,55.241446513642266,65.43866943866944,12.374220374220373,1.6292115011465866,1.3753335874952344,7.273185483870968,0.039171420726684766,-0.18578947368421053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-12,2022-12-12,AXP,154.260000,157.760000,154.000000,157.310000,2773675.0,Q3,18.060160427807485,2.5120320855614975,...,11196000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.07848788638262323,-0.5922459893048129
2022-12-13,2022-12-13,AXP,160.620000,161.550000,156.160000,157.520000,3110000.0,Q3,18.060160427807485,2.5120320855614975,...,11196000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.07848788638262323,-0.5922459893048129
2022-12-14,2022-12-14,AXP,157.570000,158.620000,153.840000,154.110000,3554804.0,Q3,18.060160427807485,2.5120320855614975,...,11196000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.07848788638262323,-0.5922459893048129
2022-12-15,2022-12-15,AXP,151.650000,152.280000,146.470000,150.220000,4282628.0,Q3,18.060160427807485,2.5120320855614975,...,11196000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.07848788638262323,-0.5922459893048129


In [15]:
dataset = dataset.sort_values(by="date")
dataset.index = dataset["date"].factorize()[0]

In [16]:
assert dataset.groupby("date").size().unique().size == 1

In [17]:
dataset

,date,tic,open,high,low,close,volume,period,revenuePerShare,netIncomePerShare,...,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare
0,2008-03-20,DIS,31.280001,31.980000,31.250000,31.900000,13802300.0,Q3,4.861052631578947,0.6757894736842105,...,0.0,1291000000,55.241446513642266,65.43866943866944,12.374220374220373,1.6292115011465866,1.3753335874952344,7.273185483870968,0.039171420726684766,-0.18578947368421053
0,2008-03-20,MRK,41.593510,41.593510,40.076336,41.374046,17209522.0,Q2,2.8220097924924223,0.8245744928887853,...,2006450000,3862800000,54.23675600647741,34.21482277121375,141.17722878625136,1.6593912805045243,2.630438877378037,0.6374965762804711,0.08729759083728278,-0.13578922825833528
0,2008-03-20,IBM,111.940727,113.269600,111.520073,113.126198,11943123.0,Q2,19.159774573666105,2.023713679279807,...,5619000000,2299500000,93.60990144396057,44.529925760462525,17.236055449707642,0.9614367562802997,2.021112734032665,5.221612349914237,0.09782762524766488,-0.8548634999634048
0,2008-03-20,MCD,53.950001,54.759998,53.700001,54.419998,13075600.0,Q2,5.381610417220303,1.0545663920630703,...,670400000,79200000,15.26739420275542,14.05687573042462,2.69029996104402,5.894915583155443,6.402560691719321,33.45351867940921,0.08142063796027794,-0.42696430153246523
0,2008-03-20,AMGN,40.160000,40.220001,39.160000,40.110001,17844400.0,Q2,3.49165120593692,0.87291280148423,...,1010000000,3344000000,55.73591923485653,133.51456310679612,372.93203883495147,1.6147576147576148,0.6740837696335078,0.24133083411433928,0.05077429450169967,-0.15306122448979592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3713,2022-12-16,UNH,525.150000,525.810000,515.720000,523.700000,7109629.0,Q3,85.96898395721925,5.627807486631016,...,55209500000,0.0,34.40850449733146,83.14008631299126,0.0,2.6156324232859327,1.0825103026858036,0.0,0.07049272566513946,-0.774331550802139
3713,2022-12-16,TRV,181.490000,183.800000,181.150000,182.800000,3364427.0,Q3,39.51996601529312,1.9286321155480033,...,0,0.0,85.96581747823282,0.0,0.0,1.0469277515192437,0,0.0,0.022807193810911282,0.0
3713,2022-12-16,HON,207.350000,210.290000,206.230000,209.740000,7129182.0,Q3,13.2784453345201,2.302329031300994,...,6181500000,5538500000,74.03306893084572,92.0615281725464,82.77712757064036,1.2156729593915523,0.9776070611310886,1.0872568623886567,0.08764895239170949,-0.2729565346387776
3713,2022-12-16,VZ,37.510000,37.570000,36.670000,37.120000,30249999.0,Q3,8.148738695859114,1.1661113755354593,...,0.0,3389500000,62.21488858386145,0.0,19.31169098006986,1.4465990705534433,0,4.660389403127993,0.05602048749256871,-1.4124226558781532


In [18]:
print(dataset["date"].min())
print(dataset["date"].max())

date_split = dataset.iloc[dataset.index.size // 4 * 3]["date"]
dataset[dataset["date"] < date_split]

2008-03-20
2022-12-16


,date,tic,open,high,low,close,volume,period,revenuePerShare,netIncomePerShare,...,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare
0,2008-03-20,DIS,31.280001,31.980000,31.250000,31.900000,13802300.0,Q3,4.861052631578947,0.6757894736842105,...,0.0,1291000000,55.241446513642266,65.43866943866944,12.374220374220373,1.6292115011465866,1.3753335874952344,7.273185483870968,0.039171420726684766,-0.18578947368421053
0,2008-03-20,MRK,41.593510,41.593510,40.076336,41.374046,17209522.0,Q2,2.8220097924924223,0.8245744928887853,...,2006450000,3862800000,54.23675600647741,34.21482277121375,141.17722878625136,1.6593912805045243,2.630438877378037,0.6374965762804711,0.08729759083728278,-0.13578922825833528
0,2008-03-20,IBM,111.940727,113.269600,111.520073,113.126198,11943123.0,Q2,19.159774573666105,2.023713679279807,...,5619000000,2299500000,93.60990144396057,44.529925760462525,17.236055449707642,0.9614367562802997,2.021112734032665,5.221612349914237,0.09782762524766488,-0.8548634999634048
0,2008-03-20,MCD,53.950001,54.759998,53.700001,54.419998,13075600.0,Q2,5.381610417220303,1.0545663920630703,...,670400000,79200000,15.26739420275542,14.05687573042462,2.69029996104402,5.894915583155443,6.402560691719321,33.45351867940921,0.08142063796027794,-0.42696430153246523
0,2008-03-20,AMGN,40.160000,40.220001,39.160000,40.110001,17844400.0,Q2,3.49165120593692,0.87291280148423,...,1010000000,3344000000,55.73591923485653,133.51456310679612,372.93203883495147,1.6147576147576148,0.6740837696335078,0.24133083411433928,0.05077429450169967,-0.15306122448979592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2784,2019-04-11,WBA,54.590000,54.799999,53.320000,53.439999,8194100.0,Q3,38.01626552368392,1.1264974172986042,...,12962000000,9197000000,18.83466797721951,46.86049082467389,32.74596506743312,4.778422434037851,1.9205944798301486,2.7484302207818514,0.04247296233373389,-0.4978569073524563
2784,2019-04-11,V,159.110001,159.300003,157.440002,157.860001,5771400.0,Q3,2.931726907630522,1.5567269076305221,...,189000000,0.0,49.62328767123288,12.784090909090908,0.0,1.813664596273292,7.04,0.0,0.08861265895127876,-0.09738955823293173
2784,2019-04-11,HD,199.899994,201.500000,199.029999,201.479996,2465100.0,Q1,23.96094459582198,2.282470481380563,...,12329500000,20791500000,7.90455251885827,53.44333102971665,80.31271596406357,11.385843763487268,1.6840267675298226,1.120619554695063,-1.1726551563229117,-0.6185286103542235
2784,2019-04-11,CVX,125.339996,126.260002,124.750000,125.989998,3248300.0,Q2,19.225995294466365,2.278663924859668,...,19855500000,7262000000,35.637750185832665,52.940124756645076,24.347808812427985,2.5254119446568866,1.7000337723741978,3.6964311940079306,0.02752645544934301,-1.8970340317530892


In [19]:
filepath = exp_dir.out.datasets.joinpath(dataset_name).with_suffix(".csv")
dataset.to_csv(filepath)
filepath

PosixPath('/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/src/rl/experiments/_3_key_metrics_dow30/out/datasets/experiment_same_bigger_fundamental.csv')